In [1]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
df = pd.read_csv('')


In [3]:
X = df.drop(columns = ['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
y = df['unemployment_rate_2010']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

In [6]:
lr = LinearRegression()
lr.fit(Z_train, y_train)

LinearRegression()

In [7]:
lr.score(Z_train, y_train), lr.score(Z_test, y_test), cross_val_score(lr, Z_train, y_train).mean()

(0.9398751531186242, 0.9382080640520137, 0.9282232543033823)